# Import Libs

In [1]:
import os

from bot import BertGoggles

import pickle as pkl
import numpy as np

import sklearn as sk
from sklearn import linear_model
from sklearn import preprocessing as pre

from logistic import Logistic

# Load Questions

In [ ]:
with open('squad_val_questions.pkl', 'rb') as file:
    questions = pkl.load(file)

### Second quarter for training Logistic Regression

In [ ]:
questions = questions[len(questions)//4:2*len(questions)//4]

### Keep Possible Questions

In [ ]:
new_questions = []

for q in questions:
    if not q['label']:
        new_questions.append(q)
        
questions = new_questions

# Score

In [ ]:
def bobo(q, true_id, answerini, bert):
    trus = []
    fals = []
    
    anserini_scores = [0]
    contexts = [0]
    
    hits = answerini.search(q, top_n=100)
    
    true_found = False
    i = 0
    
    while len(fals) < 10 or not true_found:
        hit = hits[i]
        if true_id == int(hit.id):
            true_found = True
            contexts[0] = hit
            anserini_scores[0] = hit.score
            
        elif len(contexts) < 11:
            contexts.append(hit)
            anserini_scores.append(hit.score)
            
        i += 1
        
        if i >= len(hits):
            break
    
    if true_found:
        bert_hits = bert.search(q, contexts)
        
        trus.append([anserini_scores[0], bert_hits[0].score, bert_hits[0].null_score])
        
        for i in range(1, len(bert_hits)):
            fals.append([anserini_scores[i], bert_hits[i].score, bert_hits[i].null_score])
        
    else:
        bert_hits = bert.search(q, contexts[1:])
        
        for i in range(len(bert_hits)):
            fals.append([anserini_scores[i + 1], bert_hits[i].score, bert_hits[i].null_score])
        
    return trus, fals, true_found

def upper_bobo(bert_goggles, collection):
    result = []
    trus = []
    fals = []
    not_found = []
    
    from time import time
    start_time = time()
    avg = 0
    wait_iter = 5

    for i, c in enumerate(collection):

        question = c['question']

        correct_id = c['id']

        tru, fal, found = bobo(question, correct_id, bert_goggles.answerini, bert_goggles.bert_model)

        if found:
            trus.extend(tru)
            fals.extend(fal)
        else:
            not_found.append(i)
            
        if (i + 1) % wait_iter == 0:
            from IPython.display import clear_output
            clear_output(wait=True)
            
            if avg == 0:
                avg = (time() - start_time) / wait_iter
            else:
                avg = (((time() - start_time) / wait_iter) + avg) / 2
            
            start_time = time()
            print('{0:.2f}%'.format(round(i / len(collection), 4) * 100))
            
            time_left = (len(collection) - i) * avg / 60 
            
            print('Time remaining: {0:.2f} mins'.format(time_left))
            
    return trus, fals, not_found

### Index Dir

In [ ]:
INDEX = 'Index'

### Set Dirs

In [ ]:
BERT_MODEL = 'Pret-trained/model'
BERT_MODEL

In [ ]:
BERT_CHKPT = 'Fine-truned/checkpoint'
BERT_CHKPT

### BERT length

In [ ]:
MAXLEN = 350

## Load Model

In [ ]:
bert_goggles = BertGoggles(BERT_MODEL, BERT_CHKPT, MAXLEN, INDEX, top_n=10)

## Get Results

In [ ]:
BERT_results = upper_bobo(bert_goggles, questions)

### Save Results

In [ ]:
with open('results_log.pkl', 'wb') as file:
    pkl.dump(BERT_results, file)

### Load Results

In [ ]:
with open('results_log.pkl', 'rb') as file:
    BERT_results = pkl.load(file)

In [ ]:
trus = np.asarray(BERT_results[0])
fals = np.asarray(BERT_results[1])

# Logistic Regression

In [ ]:
log = Logistic()

### Train with S_null

In [ ]:
log.train(trus, fals)

### Save model

In [ ]:
with open('log_1.pkl', 'wb') as file:
    pkl.dump(log, file)

### Train without S_null

In [ ]:
# to remove s_null
trus = trus[:, :2]
fals = fals[:, :2]

# train
log.train(trus, fals, s_null=False)

### Save Model

In [ ]:
with open('log_2.pkl', 'wb') as file:
    pkl.dump(log, file)